In [1]:
import tensorflow as tf
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
if tf.test.is_gpu_available():
    print("GPU is available")
else:
    print("GPU is NOT available")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT available


In [11]:
from tensorflow.python.client import device_lib
def get_available_devices():
     local_device_protos = device_lib.list_local_devices()
     return [x.name for x in local_device_protos]
print(get_available_devices()) 
# my output was => ['/device:CPU:0']
# good output must be => ['/device:CPU:0', '/device:GPU:0']

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [6]:
def data_to_matrix_function():
    #this function will take the raw data and column size
    #and will return the main X decision matrix as an output.

    raw_data = open("UCI_Credit_Card.csv",'r')
    #the raw data obtained from kaggle.com as below:
    #https://www.kaggle.com/datasets/uciml/default-of-credit-card-clients-dataset?resource=download

    column_size = 1000
    #column size will be input for model training with X which will have variying column size
    #there are 23 number of fixed features.
    #X1,X2,X3,X4,...,X23.
    
    predictor_matrix = [] 
    response_vector = []
     
    for i in range(1,column_size+2): #first two line are predictor names.
        
        row_vector_str = raw_data.readline()

        if row_vector_str[0].isdigit():
            row_vector_lst = row_vector_str.split(',')
            row_vector_lst[-1] = row_vector_lst[-1][0:1]
            response_vector.append(int(row_vector_lst[-1]))

            row_vector_float_lst = [1]                                                 
            for i2 in range(1,len(row_vector_lst)-1): # last value is the y value of the regression.
                 row_vector_float_lst.append(float(row_vector_lst[i2]))
            
            predictor_matrix.append(row_vector_float_lst)
        else:
            continue

    return predictor_matrix, response_vector

design,response = data_to_matrix_function()

In [7]:
def Newton_Raphson(X,Y,iteration_number):

    Beta_coefficients = tf.fill([1,24],0.5)         # B = [B0,B1,B2,B3,...B23], unknown random coefficients which
    
    Beta_new = tf.zeros([1,24],dtype=tf.float64)
    Beta_old = Beta_coefficients
    Beta_old = tf.cast(Beta_old, dtype=tf.float64)

    row_size = X.shape[0]

    #Pi vector generation:
    Pi_vector =  tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)                         
    for i2 in range(0, row_size):
        Pi_vector.write(i2,logistic_function(Beta_old,X[i2,:]))
    Pi_vector = Pi_vector.stack()

    #W matrix generation:
    W = tf.eye(X.shape[0])
    for i in range(0, X.shape[0]):
        indices = tf.constant([[i, i]])  # Index to update
        logistic_value = logistic_function(Beta_coefficients, X[i, :])* (1 - logistic_function(Beta_coefficients, X[i, :]))
        updates = updates = tf.constant([logistic_value])
        W = tf.tensor_scatter_nd_update(W, indices, updates)
        
    X_T = tf.transpose(X)

    X = tf.cast(X, dtype=tf.float64)
    W = tf.cast(W, dtype=tf.float64)
    X_T = tf.cast(X_T, dtype=tf.float64)
    Pi_vector = tf.cast(Pi_vector, dtype=tf.float64)

    A = tf.matmul(tf.matmul(X_T,W), X)  #A = X_t*W*X
    A_inverse = tf.linalg.inv(A)  #A^-1 = (X_t*W*X)^-1

    count = 0

    while count != iteration_number:

        A = tf.matmul(tf.matmul(X_T,W), X) #A = X_t*W*X
        A_inverse = tf.linalg.inv(A) #A^-1 = (X_t*W*X)^-1

        C1 = tf.matmul(A_inverse,X_T)
        C2 = Y - Pi_vector

        C1 = tf.cast(C1, dtype=tf.float64)
        C2 = tf.cast(C2, dtype=tf.float64)

        C2 = tf.reshape(C2, [-1, 1])
        C3 = tf.matmul(C1,C2)

        C3 = tf.cast(C3, dtype=tf.float64)
        Beta_new = Beta_old + C3

        #updating matrices and coefficients.
        Beta_old = Beta_new
        
        for i in range(0, X.shape[0]):
            indices = tf.constant([[i]])  # Index to update
            updates = tf.constant([logistic_function(Beta_old,X[i,:])],dtype=tf.float64)
            Pi_vector = tf.tensor_scatter_nd_update(Pi_vector, indices, updates)

        for i in range(0, X.shape[0]):
            indices = tf.constant([[i,i]])  # Index to update
            updates = tf.constant([logistic_function(Beta_old,X[i,:])],dtype=tf.float64)
            W = tf.tensor_scatter_nd_update(W, indices, updates)
            
        
        count += 1
        
        print(count)

    return Beta_new
    

def logistic_function(beta_vector,row_vector):
    beta_vector = tf.cast(beta_vector, dtype=tf.float64)
    row_vector = tf.cast(row_vector, dtype=tf.float64)
    dot_product = tf.constant([], dtype=tf.float64)

    dot_product = tf.tensordot(beta_vector, row_vector, axes=1) / 10000
    logistic_result = 1 / (1 + tf.exp(-dot_product))
    
    result = logistic_result[0].numpy().item()
    return result

In [ ]:
design,response = data_to_matrix_function()

X = tf.constant(design)
Y = tf.constant(response)
Beta_coefficients = tf.fill([1,24],0.5)

for i in range(X.shape[0]):
    row_vector = X[i,:]
    result = logistic_function(Beta_coefficients, X[i, :]) * (1 - logistic_function(Beta_coefficients, X[i, :]))
    print(type(result))


In [ ]:
design,response = data_to_matrix_function()

X = tf.constant(design)
Y = tf.constant(response)
Beta_coefficients = tf.fill([1,24],0.5)
Beta_new = tf.zeros([1,24],dtype=tf.float64)
Beta_old = Beta_coefficients

#Pi vector generation:
Pi_vector =  tf.TensorArray(dtype=tf.float64, size=0, dynamic_size=True)                         
for i2 in range(0, X.shape[0]):
    Pi_vector.write(i2,logistic_function(Beta_coefficients,X[i2,:]))
Pi_vector = Pi_vector.stack()

W = tf.eye(X.shape[0])
for i in range(0, X.shape[0]):
    indices = tf.constant([[i, i]])  # Index to update
    new_value = logistic_function(Beta_coefficients, X[i, :]) * (1 - logistic_function(Beta_coefficients, X[i, :]))
    updates = tf.constant([new_value])
    W = tf.tensor_scatter_nd_update(W, indices, updates)

X_T = tf.transpose(X)

X = tf.cast(X, dtype=tf.float64)
Y = tf.cast(Y, dtype=tf.float64)
W = tf.cast(W, dtype=tf.float64)
X_T = tf.cast(X_T, dtype=tf.float64)
Pi_vector = tf.cast(Pi_vector, dtype=tf.float64)

A = tf.matmul(tf.matmul(X_T,W), X)  #A = X_t*W*X
A_inverse = tf.linalg.inv(A)  #A^-1 = (X_t*W*X)^-1

C1 = tf.matmul(A_inverse,X_T)
C2 = Y - Pi_vector

C1 = tf.cast(C1, dtype=tf.float64)
C2 = tf.cast(C2, dtype=tf.float64)

C2 = tf.reshape(C2, [-1, 1])
C3 = tf.matmul(C1,C2)

C2 = tf.reshape(C2, [-1, 1])
C3 = tf.matmul(C1,C2)

C3 = tf.cast(C3, dtype=tf.float64)
Beta_old = tf.cast(Beta_old, dtype=tf.float64)

Beta_new = Beta_old + C3
print(Beta_new)
#updating matrices and coefficients.
Beta_old = Beta_new
        
for i in range(0, X.shape[0]):
    indices = tf.constant([[i]])  # Index to update
    updates = tf.constant([logistic_function(Beta_old,X[i,:])],dtype=tf.float64)
    Pi_vector = tf.tensor_scatter_nd_update(Pi_vector, indices, updates)

for i in range(0, X.shape[0]):
    indices = tf.constant([[i,i]])  # Index to update
    updates = tf.constant([logistic_function(Beta_old,X[i,:])],dtype=tf.float64)
    W = tf.tensor_scatter_nd_update(W, indices, updates)

#Beta_hat = Newton_Raphson(X,Y,10000)
#print(Beta_hat)

In [8]:
design,response = data_to_matrix_function()

X = tf.constant(design,dtype=tf.float64)
Y = tf.constant(response,dtype=tf.float64)

Beta_hat = Newton_Raphson(X,Y,10000)

print(Beta_hat)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


KeyboardInterrupt: 

In [123]:
import tensorflow as tf

def logistic_function(beta_vector, row_vector):
    beta_vector = tf.cast(beta_vector, dtype=tf.float64)
    row_vector = tf.cast(row_vector, dtype=tf.float64)
    dot_product = tf.constant([], dtype=tf.float64)
    
    dot_product = tf.tensordot(beta_vector, row_vector, axes=1) / 10000
    logistic_result = 1 / (1 + tf.exp(-dot_product))
    return logistic_result

# Example usage
beta_vector = tf.constant([0.1, 0.2, 0.3])
row_vector = tf.constant([1, 2, 3])
result = logistic_function(beta_vector, row_vector).numpy().item()
print(result)


0.5000350000010232


In [ ]:
import tensorflow as tf

# Define the original tensor
original_tensor = tf.eye(24)

# Define the new value
new_value = logistic_function(Beta_coefficients, X[i, :]) * (1 - logistic_function(Beta_coefficients, X[i, :]))
new_value2 = 5.1


# Loop to accumulate updates
for i in range(0, 24):
    # Create indices tensor for the element to update
    indices = tf.constant([[i,i]])
    # Create updates tensor with the new value
    updates = tf.constant([new_value],dtype=original_tensor.dtype)

    # Update the original tensor using tf.tensor_scatter_nd_update()
    original_tensor = tf.tensor_scatter_nd_update(original_tensor, indices, updates)

# Display the modified tensor
print("Original tensor:")
print(original_tensor.numpy())

